# 데이터 전처리

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn 

#시각화 패키지
import matplotlib.pyplot as plt

#시각화 할 때 한글을 출력할 수 있도록 하기 위해서
import platform
from matplotlib import font_manager, rc


#한글 처리
#매킨토시의 경우
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
#윈도우의 경우
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(
        fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)


## 수치 데이터

### 이상치 감지

In [2]:
#z-score를 이용하는 방법
#z-score: (데이터-평균)/표준편차 가 절대값으로 3이 넘는 경우 이상치
#z-score를 이용한 이상치 감지를 위한 함수
def outliers_z_score(ys):
    #임계값 설정
    threshold = 3
    #평균 구하기
    mean_y = np.mean(ys)
    #표준편차 구하기
    stdev_y = np.std(ys)
    #z_score 구하기
    z_scores = [(y - mean_y) / stdev_y for y in ys]
    #이상치를 검출한 후 인덱스를 리턴
    return np.where(np.abs(z_scores) > threshold)


features = np.array([[10,10,7,6,4,5,3,3],
                    [15000,10,7,6,4,2,2,2]])
print(outliers_z_score(features))



(array([1], dtype=int64), array([0], dtype=int64))


In [4]:
#z-score를 보정 - MAD(중위 절대 편차)
def outliers_modified_z_score(ys):
    #임계값 설정
    threshold = 3.5
    #중앙값 구하기
    median_y = np.median(ys)
    #편차값 구하기
    median_absolute_deviation_y = np.median([np.abs(y-median_y) for y in ys])
    #보정한 z_score 구하기
    median_z_scores = [0.6745 * (y - median_y) / median_absolute_deviation_y  for y in ys]
    #이상치를 검출한 후 인덱스를 리턴
    return np.where(np.abs(median_z_scores) > threshold)

features = np.array([[10,10,7,6,4,5,3,3],
                    [15000,10,7,6,4,2,2,2]])
print(outliers_modified_z_score(features))

(array([1], dtype=int64), array([0], dtype=int64))


In [5]:
#IQR을 이용하는 방법 - (3사분위수-1사분위수)
#1사분위수에서 IQR*1.5 한 값을 뺀 것보다 적거나 
#3사분위수에서 IQR*1.5 한 값을 더한 것보다 큰 경우
def outliers_iqr(ys):
    #1사분위수 와 3사분위수 구하기
    q1, q3 = np.percentile(ys, [25, 75])
    iqr = q3 - q1
    #경계값 계산
    lower_bound = q1 - (iqr * 1.5)
    upper_bound = q3 + (iqr * 1.5)
    print("하한:", lower_bound)
    print("상한:", upper_bound)
    return np.where((ys < lower_bound) | (ys > upper_bound))
    
features = np.array([[10,10,7,6,4,5,3,3],
                    [15000,10,7,6,4,2,2,2]])
print(outliers_iqr(features))

하한: -4.125
상한: 14.875
(array([1], dtype=int64), array([0], dtype=int64))


In [10]:
#일정 비율을 이상치로 설정
from sklearn.covariance import EllipticEnvelope

#10퍼센트를 이상치로 간주하는 객체 생성
outlier_detector = EllipticEnvelope(contamination = 0.1)

#샘플 데이터 생성
from sklearn.datasets import make_blobs
features, _ = make_blobs(n_samples = 10,
                        n_features = 2,
                        centers = 1,
                        random_state = 42)
#1번 행의 데이터를 이상치로 변경
features[1,0] = 2000
features[1,1] = 1500
print(features)

#훈련
outlier_detector.fit(features)
#예측
outlier_detector.predict(features)

[[-2.74335100e+00  8.78014917e+00]
 [ 2.00000000e+03  1.50000000e+03]
 [-3.52202874e+00  9.32853346e+00]
 [-2.26723535e+00  7.10100588e+00]
 [-2.97261532e+00  8.54855637e+00]
 [-1.04354885e+00  8.78850983e+00]
 [-1.86150908e+00  1.05373160e+01]
 [-2.97867201e+00  9.55684617e+00]
 [-4.23411546e+00  8.45199860e+00]
 [-9.29984808e-01  9.78172086e+00]]


array([ 1, -1,  1,  1,  1,  1,  1,  1,  1,  1])

### 이상치 처리를 위한 특성 변환

In [18]:
houses = pd.DataFrame()
houses['price'] = [534433, 392333, 203222]
houses['bathrooms'] = [2, 3, 200]
houses['square_feet'] = [1500, 2500, 2000]

#print(houses)

#새로운 특성 추가 - bathrooms 가 20이 넘는 경우 새로운 특성으로 변환
houses['outlier'] = np.where(houses['bathrooms'] < 20, 0, 1)
#print(houses)

#변환 - 로그 변환
houses['log_square_feet'] = [np.log(x) for x in houses['square_feet']]
#print(houses)

#스케일링
#이상치에 덜 민감한 스케일링
scaler = sklearn.preprocessing.RobustScaler()
scaler.fit(houses[['bathrooms']])
print(scaler.transform(houses[['bathrooms']]))
#이상치에 민감한 스케일링
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(houses[['bathrooms']])
print(scaler.transform(houses[['bathrooms']]))

scaler = sklearn.preprocessing.MinMaxScaler()
scaler.fit(houses[['bathrooms']])
print(scaler.transform(houses[['bathrooms']]))

[[-0.01010101]
 [ 0.        ]
 [ 1.98989899]]
[[-0.71247036]
 [-0.7017296 ]
 [ 1.41419997]]
[[0.        ]
 [0.00505051]
 [1.        ]]


### 결측치 확인

In [24]:
#seaborn 의 titanic 데이터 가져오기
titanic = sns.load_dataset('titanic')

#RangeIndex 의 개수 와 열의 데이터 개수가 맞지 않으면 결측치가 존재
#titanic.info()

#각 데이터의 개수를 리턴 - 옵션이 없으면 None 은 제외
#print(titanic['deck'].value_counts())

#None 을 제외하지 않고 리턴
#print(titanic['deck'].value_counts(dropna = False))

#isnull 이용
print(titanic['deck'].isnull().sum(axis=0))

688


### 결측치 처리

In [29]:
#결측치 제거
features = np.array([
    [1.1, 11.1], 
    [2.1, 21.1], 
    [3.1, 31.1], 
    [4.1, 41.1], 
    [np.nan, 51.1]])
#print(features)

#~ 연산자를 이용한 None 제거
#isnan 이 True 인 데이터를 제외하고 가져오기
print(features[~np.isnan(features).any(axis=1)])

#None 의 개수가 500개 이상인 열 제거
titanic_thresh = titanic.dropna(axis=1, thresh=500)
titanic_thresh.info()

#age 가 None 인 데이터 삭제
titanic_age = titanic_thresh.dropna(subset=['age'], how='any')
titanic_age.info()

[[ 1.1 11.1]
 [ 2.1 21.1]
 [ 3.1 31.1]
 [ 4.1 41.1]]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  embark_town  889 non-null    object  
 12  alive        891 non-null    object  
 13  alone        891 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(5)
memory usage: 79.4+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 8

In [33]:
#결측치 대체
#829 번째 데이터에 NaN이 있음
print(titanic['embark_town'][825:835])

#이전 값으로 채우기
#ffill 대신에 bfill 가능
#method를 제거하고 다른 값으로 작성해도 됨
titanic['embark_town'].fillna(method='ffill', inplace=True)
print(titanic['embark_town'][825:835])

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829            NaN
830      Cherbourg
831    Southampton
832      Cherbourg
833    Southampton
834    Southampton
Name: embark_town, dtype: object
825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829     Queenstown
830      Cherbourg
831    Southampton
832      Cherbourg
833    Southampton
834    Southampton
Name: embark_town, dtype: object


In [35]:
#가장 자주 등장하는 데이터로 채우기
titanic = sns.load_dataset('titanic')

#embark_town 의 값의 개수를 확인
mode = titanic['embark_town'].value_counts()
#print(mode)

#결측치를 가장 빈번히 등장하는 단어로 채우기
titanic['embark_town'].fillna(mode.idxmax(), inplace=True)
print(titanic['embark_town'][825:835])

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829    Southampton
830      Cherbourg
831    Southampton
832      Cherbourg
833    Southampton
834    Southampton
Name: embark_town, dtype: object


In [40]:
#중간값으로 대체 - SimpleImputer 이용
from sklearn.impute import SimpleImputer

#None 데이터 확인
print(titanic['age'][880:890])
#중간값 확인
print(titanic['age'].median())

#중간값으로 대체
simple_imputer = SimpleImputer(strategy='median')
result = simple_imputer.fit_transform(titanic[['age']])
#채워진 데이터 확인
print(result[888])

880    25.0
881    33.0
882    22.0
883    28.0
884    25.0
885    39.0
886    27.0
887    19.0
888     NaN
889    26.0
Name: age, dtype: float64
28.0
[28.]


In [43]:
#머신러닝으로 값을 유추해서 채우기
from fancyimpute import KNN

features = np.array([
    [100,200], [200, 400], [150, 300], [190, 380], [200, np.nan]])

print(KNN(k=5, verbose=0).fit_transform(features))

[[100.         200.        ]
 [200.         400.        ]
 [150.         300.        ]
 [190.         380.        ]
 [200.         400.00000394]]


## 범주형 데이터 - category

### one hot encoding

In [54]:
mpg = pd.read_csv('./data/auto-mpg.csv', header=None)

#열이름 설정
mpg.columns = ['mpg', 'cylinders', 'displacement', 'horsepower',
              'weight', 'acceleration', 'model year', 'origin',
              'name']
#mpg.info()

#horsepower 를 3개의 구간으로 분할

#실수로 변환하기 위해서 ?를 None으로 치환
mpg['horsepower'].replace('?', np.nan, inplace=True)
#horsepower 가 None 인 행을 삭제
mpg.dropna(subset=['horsepower'], axis=0, inplace=True)
#실수로 변환
mpg['horsepower'] = mpg['horsepower'].astype('float')
#mpg.info()


#구간의 이름을 생성
bin_names = ['저출력', '보통출력', '고출력']
#구간의 경계값을 생성
count, bin_dividers = np.histogram(mpg['horsepower'], bins=3)
#print(bin_dividers)


#구간 분할
mpg['hp_bin'] = pd.cut(x=mpg['horsepower'],
                      bins=bin_dividers,
                      labels=bin_names,
                      include_lowest=True)
#print(mpg)

#원핫인코딩
#범주형 데이터의 개수만큼 컬럼이 생성되고
#자신의 값에 해당하는 컬럼에만 1을 대입하고 나머지는 0
horsepower_dummies = pd.get_dummies(mpg['hp_bin'])
print(horsepower_dummies)

     저출력  보통출력  고출력
0      0     1    0
1      0     1    0
2      0     1    0
3      0     1    0
4      0     1    0
..   ...   ...  ...
393    1     0    0
394    1     0    0
395    1     0    0
396    1     0    0
397    1     0    0

[392 rows x 3 columns]


In [55]:
#sklearn 을 이용한 원 핫 인코딩

from sklearn import preprocessing

one_hot = preprocessing.LabelBinarizer()
print(one_hot.fit_transform(mpg['hp_bin']))

[[0 1 0]
 [0 1 0]
 [0 1 0]
 ...
 [0 0 1]
 [0 0 1]
 [0 0 1]]


In [59]:
#여러 개의 특성을 원 핫 인코딩
multiclass_feature = [("Python", "Java"), 
                     ("C++", "Python"), 
                     ("C++", "Java"),
                     ("Java", "Javascript"),
                     ("Python", "Javascript")]

one_hot_multiclass = preprocessing.MultiLabelBinarizer()
print(one_hot_multiclass.fit_transform(multiclass_feature))
print(one_hot_multiclass.classes_)



[[0 1 0 1]
 [1 0 0 1]
 [1 1 0 0]
 [0 1 1 0]
 [0 0 1 1]]
['C++' 'Java' 'Javascript' 'Python']


In [60]:
#하나의 컬럼에 일련번호 형태로 원 핫 인코딩
label_encoder =  preprocessing.LabelEncoder()
print(label_encoder.fit_transform(mpg['hp_bin']))

[1 1 1 1 1 0 0 0 0 0 0 1 1 0 2 2 2 2 2 2 2 2 2 1 2 0 0 0 0 2 2 2 2 2 2 2 2
 1 0 1 1 0 0 0 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 0 1 1 1 0 1 1 0 2 1 1 1
 1 1 2 2 2 2 2 2 2 2 0 1 1 1 1 0 1 1 1 0 0 0 2 2 2 2 2 2 1 1 0 0 2 2 2 2 2
 2 2 2 1 0 2 2 2 1 1 1 1 0 2 2 2 2 2 2 2 2 1 2 1 1 1 1 1 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 0 1 1 1 1 2 1 2 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2
 1 1 1 1 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 1 2 1 1 0 1 1 1 2 2 2 2 2 1 1 1
 1 1 2 2 2 0 0 0 1 2 2 2 2 2 2 2 2 2 1 1 2 2 2 2 2 1 1 1 2 2 2 2 2 2 2 2 1
 1 1 1 1 1 2 2 2 2 2 2 2 2 2 1 1 1 2 2 1 2 2 2 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 1 2 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2
 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 1 2 2 1 2 2 2 2 2 2 2 2]


In [62]:
#희소행렬로 표현 - 0이 아주 많은 행렬이라서 0이 아닌 좌표만 가지고 있습니다.

features = [["Java", 1], ["C++", 2], ["C#", 1], ["Python", 2]]

onehot_encoder = preprocessing.OneHotEncoder()
print(onehot_encoder.fit_transform(features))

#밀집행렬로 표현 - 원핫 인코딩 과 유사한 결과
onehot_encoder = preprocessing.OneHotEncoder(sparse=False)
print(onehot_encoder.fit_transform(features))

  (0, 2)	1.0
  (0, 4)	1.0
  (1, 1)	1.0
  (1, 5)	1.0
  (2, 0)	1.0
  (2, 4)	1.0
  (3, 3)	1.0
  (3, 5)	1.0
[[0. 0. 1. 0. 1. 0.]
 [0. 1. 0. 0. 0. 1.]
 [1. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 1.]]


### 순서가 있는 경우의 변환

In [64]:
df = pd.DataFrame({"점수":["저조", "보통", "우수", "보통", "저조"]})
#print(df)

scale_mapper = {"저조":1, "보통":2, "우수":3}

df['encoder'] = df['점수'].replace(scale_mapper)
print(df)

   점수  encoder
0  저조        1
1  보통        2
2  우수        3
3  보통        2
4  저조        1


In [66]:
# 문자열이나 숫자(문자로 취급)를 정렬해서 인코딩
features = np.array([["Low", 10], 
                    ["High", 3],
                    ["Medium", 27]])

ordinal_encoder = preprocessing.OrdinalEncoder()
print(ordinal_encoder.fit_transform(features))

[[1. 0.]
 [0. 2.]
 [2. 1.]]


### Dictionary를 특성 행렬로 변환

In [69]:
data_dict = [{"Red":2, "Blue":4}, 
            {"Red":4, "Blue":3},
            {"Red":1, "Yellow": 2},
            {"Red":2, "Yellow": 2}]

from sklearn.feature_extraction import DictVectorizer

dictVectorizer = DictVectorizer(sparse=False)
print(dictVectorizer.fit_transform(data_dict))

dictVectorizer.get_feature_names()

[[4. 2. 0.]
 [3. 4. 0.]
 [0. 1. 2.]
 [0. 2. 2.]]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['Blue', 'Red', 'Yellow']

### 결측치 대체

In [71]:
#데이터 생성
X = np.array([[0, 2.10, 1.45], 
             [1, 1.18, 1.33],
             [0, 1.22, 1.27], 
             [1, -0.22, -1.19]])

y = np.array([[np.nan, 0.87, 1.31],
             [np.nan, 0.67, 0.22]])

print(X)
print(y)

[[ 0.    2.1   1.45]
 [ 1.    1.18  1.33]
 [ 0.    1.22  1.27]
 [ 1.   -0.22 -1.19]]
[[ nan 0.87 1.31]
 [ nan 0.67 0.22]]


In [76]:
# 머신러닝으로 대체
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(3, weights='distance')

#1번열 이후의 데이터를 가지고 0번 열을 예측하는 모델을 생성
train_model = clf.fit(X[:, 1:], X[:, 0])
#예측
imputed_values = train_model.predict(y[:, 1:])
print(imputed_values)

#대체
#imputed_values 를 2차원으로 변환해서 합치기
y_with_imputed = np.hstack(
    (imputed_values.reshape(-1, 1), y[:, 1:]))
print(y_with_imputed)

#대체된 데이터와 X 를 위아래로 합치기
np.vstack((X, y_with_imputed))

[0. 1.]
[[0.   0.87 1.31]
 [1.   0.67 0.22]]


array([[ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.22, -1.19],
       [ 0.  ,  0.87,  1.31],
       [ 1.  ,  0.67,  0.22]])

In [82]:
#가장 많이 등장하는 데이터로 대체
X_complete = np.vstack((X, y))
print(X_complete)

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')
imputer.fit_transform(X_complete)

[[ 0.    2.1   1.45]
 [ 1.    1.18  1.33]
 [ 0.    1.22  1.27]
 [ 1.   -0.22 -1.19]
 [  nan  0.87  1.31]
 [  nan  0.67  0.22]]


array([[ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.22, -1.19],
       [ 0.  ,  0.87,  1.31],
       [ 0.  ,  0.67,  0.22]])

### 불균형한 데이터

In [84]:
#가중치를 설정해서 모델을 생성
from sklearn.ensemble import RandomForestClassifier

#가중치를 설정
weights = {0:0.9, 1:0.1}
#모델 생성시 설정
rfc = RandomForestClassifier(class_weight=weights)
print(rfc)

rfc = RandomForestClassifier(class_weight="balanced")
print(rfc)

RandomForestClassifier(class_weight={0: 0.9, 1: 0.1})
RandomForestClassifier(class_weight='balanced')


In [91]:
# 다운 샘플링 과 업 샘플링
# 0번은 10개 1번은 100개
li1 = []
for i in range(0,10, 1):
    li1.append(0)
    
li2 = []
for i in range(0,100,1):
    li2.append(1)
    
target = np.array(li1 + li2)
#print(target)

#0 번과 1번의 인덱스를 추출
i_class0 = np.where(target == 0)[0]
i_class1 = np.where(target == 1)[0]
#print(i_class0)
#print(i_class1)

#데이터의 개수 확인
n_class0 = len(i_class0)
n_class1 = len(i_class1)

#다운 샘플링 - 랜덤하게 추출
i_class1_downsampled = np.random.choice(i_class1, size=n_class0,
                                       replace = False)
#print(i_class1_downsampled)

np.hstack((target[i_class0], target[i_class1_downsampled]))

#업 샘플링
i_class0_upsampled = np.random.choice(i_class0, size=n_class1,
                                       replace = True)
np.hstack((target[i_class0_upsampled], target[i_class1]))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

## 텍스트 데이터

### 파이썬의 str 클래스에서 제공하는 함수를 이용한 가공

In [96]:
text_data = ['I am a boy.', 
             ' You are a girl.',
             'He is a Programmer.']
print(text_data)
#좌우 공백 문자를 제거
strip_text = [string.strip() for string in text_data]
print(strip_text)

#모두 대문자로 만들기
upper_text = [string.upper() for string in strip_text]
print(upper_text)

#.을 삭제
replace_text = [string.replace('.', '') for string in upper_text]
print(replace_text)

replace_text = [string.replace(' ', '') for string in replace_text]
print(replace_text)

['I am a boy.', ' You are a girl.', 'He is a Programmer.']
['I am a boy.', 'You are a girl.', 'He is a Programmer.']
['I AM A BOY.', 'YOU ARE A GIRL.', 'HE IS A PROGRAMMER.']
['I AM A BOY', 'YOU ARE A GIRL', 'HE IS A PROGRAMMER']
['IAMABOY', 'YOUAREAGIRL', 'HEISAPROGRAMMER']
